In [4]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [2]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [11]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

def get_minibatch(doc_stream, size):
    X, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            X.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return X, y

In [12]:
vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1)

doc_stream = stream_docs(path='movie_data.csv')

In [13]:
from sklearn.linear_model import SGDClassifier
import pyprind
import numpy as np

# Initialize the classifier with the correct loss parameter
clf = SGDClassifier(loss='log_loss')

# Training
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if X_train is None or y_train is None:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

# Testing
X_test, y_test = get_minibatch(doc_stream, size=5000)
if X_test is not None and y_test is not None:
    X_test = vect.transform(X_test)
    print(f'Accuracy: {clf.score(X_test, y_test):.3f}')

    # Final update
    clf = clf.partial_fit(X_test, y_test)
else:
    print("No test data available.")

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Accuracy: 0.867
